In [28]:
#　ライブラリーのインポート
import pandas as pd
import random
import numpy as np

In [29]:
# 各野菜の価格の率表

cucumber_rate = {
    "cucumber_No2": 62,
    "cucumber_No3": 60.5,
    "cucumber_No4": 54.5,
    "cucumber_No5": 75,
    "cucumber_No6": 59.5,
    "cucumber_No7": 60.5,
    "cucumber_No8": 52.5,
}

carrot_rate = {
    "carrot_No2": 65.5,
    "carrot_No3": 68,
    "carrot_No4": 55.5,
    "carrot_No5": 60.5,
    "carrot_No6": 58.5,
    "carrot_No7": 65.5,
    "carrot_No8": 55.5,
}

tomato_rate = {
    "tomato_No2": 75.5,
    "tomato_No3": 77.5,
    "tomato_No4": 73,
    "tomato_No5": 78.5,
    "tomato_No6": 73,
    "tomato_No7": 74.5,
    "tomato_No8": 68,
}

In [30]:
# 在庫の生成

# ランダムな在庫を生成
# きゅうりの在庫数
cucumber_stock = {key: random.randint(0, 100) for key in cucumber_rate}

# にんじんの在庫数
carrot_stock = {key: random.randint(0, 100) for key in carrot_rate}

# とまとの在庫数
tomato_stock = {key: random.randint(0, 100) for key in tomato_rate}

In [31]:
# きゅうりのデータフレームを作成
cucumber_df = pd.DataFrame({
    'Cucumber_Num': list(cucumber_rate.keys()),
    'Cucumber_Price': list(cucumber_rate.values()),
    'Cucumber_Stock': [cucumber_stock[key] for key in cucumber_rate.keys()]
})

# にんじんのデータフレームを作成
carrot_df = pd.DataFrame({
    'Carrot_Num': list(carrot_rate.keys()),
    'Carrot_Price': list(carrot_rate.values()),
    'Carrot_Stock': [carrot_stock[key] for key in carrot_rate.keys()]
})

# とまとのデータフレームを作成
tomato_df = pd.DataFrame({
    'Tomato_Num': list(tomato_rate.keys()),
    'Tomato_Price': list(tomato_rate.values()),
    'Tomato_Stock': [tomato_stock[key] for key in tomato_rate.keys()]
})

In [32]:
print(cucumber_df)
print(carrot_df)
print(tomato_df)

   Cucumber_Num  Cucumber_Price  Cucumber_Stock
0  cucumber_No2            62.0              47
1  cucumber_No3            60.5              70
2  cucumber_No4            54.5              57
3  cucumber_No5            75.0              22
4  cucumber_No6            59.5              38
5  cucumber_No7            60.5              20
6  cucumber_No8            52.5               0
   Carrot_Num  Carrot_Price  Carrot_Stock
0  carrot_No2          65.5            61
1  carrot_No3          68.0            19
2  carrot_No4          55.5            72
3  carrot_No5          60.5            91
4  carrot_No6          58.5            58
5  carrot_No7          65.5            61
6  carrot_No8          55.5            61
   Tomato_Num  Tomato_Price  Tomato_Stock
0  tomato_No2          75.5            47
1  tomato_No3          77.5            98
2  tomato_No4          73.0            88
3  tomato_No5          78.5            43
4  tomato_No6          73.0            79
5  tomato_No7          74.5 

In [33]:
# 初期在庫を保存
initial_cucumber_stock = cucumber_stock.copy()
initial_carrot_stock = carrot_stock.copy()
initial_tomato_stock = tomato_stock.copy()

In [34]:
# それぞれの在庫の合計を求める
cucumber_total_stock = cucumber_df['Cucumber_Stock'].sum()
carrot_total_stock = carrot_df['Carrot_Stock'].sum()
tomato_total_stock = tomato_df['Tomato_Stock'].sum()

print(cucumber_total_stock)
print(carrot_total_stock)
print(tomato_total_stock)

254
423
480


In [35]:
# 作成する箱の数
cucumber_max_box_num = cucumber_total_stock // 3
carrot_max_box_num = carrot_total_stock // 3
tomato_max_box_num = tomato_total_stock // 3

max_box_num = min(cucumber_max_box_num, carrot_max_box_num, tomato_max_box_num)

print(max_box_num)

84


In [36]:
# 各箱に含まれる野菜
cucumber_in_num = cucumber_total_stock // max_box_num
carrot_in_num = carrot_total_stock // max_box_num
tomato_in_num = tomato_total_stock // max_box_num

In [37]:
# 各箱に含まれる野菜の量
print(cucumber_in_num)
print(carrot_in_num)
print(tomato_in_num)

3
5
5


In [38]:
# 各箱を初期化
boxes = [{'Cucumber': [], 'Carrot': [], 'Tomato': [], 'Total Amount': 0} for _ in range(max_box_num)]

In [39]:
# 価格順にソート
cucumber_df = cucumber_df.sort_values(by='Cucumber_Price', ascending=False)
carrot_df = carrot_df.sort_values(by='Carrot_Price', ascending=False)
tomato_df = tomato_df.sort_values(by='Tomato_Price', ascending=False)

In [40]:
# 割り当て関数
def allocate_vegetables(df, vegetable_type, in_num, boxes):
    remaining_stock = []
    for _, row in df.iterrows():
        stock = row[f'{vegetable_type}_Stock']
        price = row[f'{vegetable_type}_Price']
        veg_name = row[f'{vegetable_type}_Num']
        for _ in range(stock):  # 在庫分だけループ
            # 割り付ける箱を選択（合計価格が最も低い箱を選ぶ）
            target_box = min(boxes, key=lambda x: x['Total Amount'])
            if len(target_box[vegetable_type]) < in_num:  # 各箱の割り付け量を制限
                target_box[vegetable_type].append((veg_name, price))
                target_box['Total Amount'] += price
            else:
                remaining_stock.append((veg_name, price))
    return remaining_stock

In [41]:
# 各野菜を割り当て
remaining_cucumber = allocate_vegetables(cucumber_df, 'Cucumber', cucumber_in_num, boxes)
remaining_carrot = allocate_vegetables(carrot_df, 'Carrot', carrot_in_num, boxes)
remaining_tomato = allocate_vegetables(tomato_df, 'Tomato', tomato_in_num, boxes)

In [42]:
# 箱の中身を表す表の作成
# 最大の野菜の割り付け数を取得
max_veg_count = max(len(box['Cucumber']) + len(box['Carrot']) + len(box['Tomato']) for box in boxes)

In [43]:
# カラム名を動的に作成
columns = ['Box_No', 'Total Amount'] + [f'veg{i+1}' for i in range(max_veg_count)]
Box_df = pd.DataFrame(columns=columns)

In [44]:
# 箱ごとにデータを追加
for i, box in enumerate(boxes):
    veg_prices = box['Cucumber'] + box['Carrot'] + box['Tomato']  # 各野菜の価格を統合
    row = [i + 1, box['Total Amount']] + veg_prices  # Box_No と Total Amount を追加
    # veg 列に不足がある場合は 0 を補充
    row += [0] * (max_veg_count - len(veg_prices))
    # DataFrame に追加
    Box_df.loc[i] = row

In [45]:
# 箱の内容を種類ごとに集計
def summarize_box_contents(boxes):
    summaries = []
    for i, box in enumerate(boxes):
        summary = {'Box_No': i + 1, 'Total Amount': box['Total Amount']}
        for veg_type in ['Cucumber', 'Carrot', 'Tomato']:
            counts = {}
            for name, price in box[veg_type]:
                counts[name] = counts.get(name, 0) + 1
            summary[veg_type] = counts
        summaries.append(summary)
    return summaries

In [46]:
# 余った在庫を種類ごとに集計
def summarize_remaining_stock(remaining_stock):
    counts = {}
    for name, price in remaining_stock:
        counts[name] = counts.get(name, 0) + 1
    return counts

In [47]:
# 使用された数をカウントする関数を修正
def count_used_stock(boxes, veg_type):
    counts = {}
    for box in boxes:
        for name, _ in box[veg_type]:
            counts[name] = counts.get(name, 0) + 1
    return counts

In [48]:
# 使用後の在庫を確認
def check_inventory_usage(initial_stock, used_stock, remaining_stock):
    check_result = {}
    for key, initial_count in initial_stock.items():
        used_count = used_stock.get(key, 0)
        remaining_count = remaining_stock.get(key, 0)
        if initial_count != used_count + remaining_count:
            check_result[key] = (initial_count, used_count, remaining_count)
    return check_result

In [49]:
# 各箱で使用された野菜の数を集計
used_cucumber = count_used_stock(boxes, 'Cucumber')
used_carrot = count_used_stock(boxes, 'Carrot')
used_tomato = count_used_stock(boxes, 'Tomato')

In [50]:
# 箱の内容を集計
box_summaries = summarize_box_contents(boxes)

# 余った在庫を集計
remaining_cucumber_summary = summarize_remaining_stock(remaining_cucumber)
remaining_carrot_summary = summarize_remaining_stock(remaining_carrot)
remaining_tomato_summary = summarize_remaining_stock(remaining_tomato)

In [51]:
# 初期在庫、使用された在庫、余った在庫を比較
cucumber_check = check_inventory_usage(initial_cucumber_stock, used_cucumber, remaining_cucumber_summary)
carrot_check = check_inventory_usage(initial_carrot_stock, used_carrot, remaining_carrot_summary)
tomato_check = check_inventory_usage(initial_tomato_stock, used_tomato, remaining_tomato_summary)

In [52]:
# 結果を表示
print("\n--- Inventory Check ---")
print("Cucumber Check:", cucumber_check)
print("Carrot Check:", carrot_check)
print("Tomato Check:", tomato_check)


--- Inventory Check ---
Cucumber Check: {}
Carrot Check: {}
Tomato Check: {}


In [53]:
# 結果を表示
print("\n--- Box Summaries ---")
for summary in box_summaries:
    print(f"Box {summary['Box_No']}:")
    print(f"  Total Amount: {summary['Total Amount']}")
    for veg_type, counts in summary.items():
        if veg_type not in ['Box_No', 'Total Amount']:
            print(f"  {veg_type}: {counts}")

print("\n--- Remaining Stock Summaries ---")
print("Remaining Cucumber:", remaining_cucumber_summary)
print("Remaining Carrot:", remaining_carrot_summary)
print("Remaining Tomato:", remaining_tomato_summary)


--- Box Summaries ---
Box 1:
  Total Amount: 860.0
  Cucumber: {'cucumber_No5': 1, 'cucumber_No7': 1, 'cucumber_No4': 1}
  Carrot: {'carrot_No2': 1, 'carrot_No5': 1, 'carrot_No6': 1, 'carrot_No4': 1, 'carrot_No8': 1}
  Tomato: {'tomato_No3': 1, 'tomato_No2': 2, 'tomato_No4': 2}
Box 2:
  Total Amount: 860.0
  Cucumber: {'cucumber_No5': 1, 'cucumber_No7': 1, 'cucumber_No4': 1}
  Carrot: {'carrot_No2': 1, 'carrot_No5': 1, 'carrot_No6': 1, 'carrot_No4': 1, 'carrot_No8': 1}
  Tomato: {'tomato_No3': 1, 'tomato_No2': 2, 'tomato_No4': 2}
Box 3:
  Total Amount: 860.0
  Cucumber: {'cucumber_No5': 1, 'cucumber_No7': 1, 'cucumber_No4': 1}
  Carrot: {'carrot_No2': 1, 'carrot_No5': 1, 'carrot_No6': 1, 'carrot_No4': 1, 'carrot_No8': 1}
  Tomato: {'tomato_No3': 1, 'tomato_No2': 2, 'tomato_No4': 2}
Box 4:
  Total Amount: 860.0
  Cucumber: {'cucumber_No5': 1, 'cucumber_No7': 1, 'cucumber_No4': 1}
  Carrot: {'carrot_No2': 1, 'carrot_No5': 1, 'carrot_No6': 1, 'carrot_No4': 1, 'carrot_No8': 1}
  Tomato: {

In [54]:
# 各箱の合計価格の分散を計算
total_amounts = [box['Total Amount'] for box in boxes]
variance = np.var(total_amounts)
print(f"Total Amount Variance: {variance}")

Total Amount Variance: 0.1696074263038549
